# Unit 7 — Object-Oriented Programming (OOP): Foundations

**Purpose:** Introduce structured program design.
---

## Why OOP now?

Up to Unit 6, you mostly wrote code in a *procedural* style:
- data is stored in lists/dicts,
- functions operate on that data.

That works well for small programs, but as systems grow, you typically want:
- a clear *owner* for data,
- predictable state changes,
- reusable building blocks that model the real world.

OOP helps by combining **data (state)** and **functions (behavior)** in one place: the **object**.

## Learning goals

By the end of Unit 7 you should be able to:

1. Explain **class vs. object**
2. Define classes with **attributes** and **methods**
3. Initialize objects with `__init__`
4. Understand **state** and how methods change it
5. Model domains like **Person/Account** and **Product/Order**
6. Use **basic inheritance** to reuse and extend behavior
7. Recognize common beginner mistakes in OOP

---

# 1) Mental model: Class vs. Object

- A **class** is a *blueprint* (definition).
- An **object** is an *instance* created from that blueprint.

Analogy:
- Class = architectural plan
- Object = actual building

In [ ]:
class Person:
    pass

p1 = Person()
p2 = Person()

print("p1:", p1)
print("p2:", p2)
print("Same object?", p1 is p2)

### What you should notice

- `p1` and `p2` are separate objects.
- They have the same class (`Person`) but different identities.

In [ ]:
print(type(p1))
print(isinstance(p1, Person))

---

# 2) Attributes: object-specific data

Attributes are variables that *belong to an object*.  
You access them using dot notation:

```python
obj.attribute
```

Objects can have different values for the same attribute.

In [ ]:
class Person:
    pass

alice = Person()
bob = Person()

alice.name = "Alice"
bob.name = "Bob"

print(alice.name)
print(bob.name)

### Common beginner error: using an attribute before setting it

If you try to access `alice.age` without setting it, you get an `AttributeError`.

In [ ]:
# Uncomment to see the error:
# print(alice.age)

---

# 3) Methods: behavior on objects

A **method** is a function defined inside a class.  
It usually works with the object's attributes.

Key point:
- The first parameter is `self` (the current object).

In [ ]:
class Person:
    def greet(self):
        print("Hello!")

p = Person()
p.greet()

### What is `self`?

When you call:

```python
p.greet()
```

Python internally does something like:

```python
Person.greet(p)
```

So `self` is just the object being acted on.

In [ ]:
class Counter:
    def increment(self):
        # create or update an attribute on this object
        if not hasattr(self, "value"):
            self.value = 0
        self.value += 1

c1 = Counter()
c2 = Counter()

c1.increment()
c1.increment()
c2.increment()

print("c1:", c1.value)
print("c2:", c2.value)

---

# 4) Constructors: `__init__`

The constructor runs automatically when an object is created.

Why it matters:
- Ensures objects start in a valid state
- Avoids missing attributes

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        print(f"Hi, I'm {self.name} ({self.age})")

alice = Person("Alice", 30)
bob = Person("Bob", 25)

alice.greet()
bob.greet()

### Default values in constructors

You can provide default values for optional parameters.

In [ ]:
class Person:
    def __init__(self, name, age=0):
        self.name = name
        self.age = age

p = Person("Newborn")
p.greet()

---

# 5) State and behavior (important!)

- **State**: attribute values at a given moment (e.g., `balance=500`)
- **Behavior**: methods that change state (e.g., `deposit()`)

We'll build a BankAccount example with validation.

In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdraw amount must be positive")
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance

    def __repr__(self):
        return f"BankAccount(owner={self.owner!r}, balance={self.balance:.2f})"

acc = BankAccount("Alice", 500)
print(acc)
acc.deposit(200)
print(acc)
acc.withdraw(100)
print(acc)

### Try error cases (learning via feedback)

Run these one by one to see the error messages.

In [ ]:
# Uncomment each line to test:
# acc.deposit(-10)
# acc.withdraw(99999)
# acc.withdraw(-1)

---

# 6) Designing with responsibilities

Ask: **Where should a function live?**

Example: A transfer between accounts can be:
- a method on `BankAccount`, or
- a function elsewhere.

We'll implement it as a method (common approach).

In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.balance += amount

    def withdraw(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdraw amount must be positive")
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

    def transfer_to(self, other, amount):
        if not isinstance(other, BankAccount):
            raise TypeError("other must be a BankAccount")
        self.withdraw(amount)
        other.deposit(amount)

    def __repr__(self):
        return f"BankAccount(owner={self.owner!r}, balance={self.balance:.2f})"

a = BankAccount("Alice", 300)
b = BankAccount("Bob", 50)

a.transfer_to(b, 100)
print(a)
print(b)

---

# 7) Composition: objects containing other objects

In many real systems, objects *contain* other objects.

Example: A Person can have multiple BankAccounts.
This is called **composition**.

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name
        self.accounts = []

    def add_account(self, account):
        if not isinstance(account, BankAccount):
            raise TypeError("account must be a BankAccount")
        self.accounts.append(account)

    def total_balance(self):
        return sum(acc.balance for acc in self.accounts)

    def __repr__(self):
        return f"Person(name={self.name!r}, accounts={len(self.accounts)})"

alice = Person("Alice")
alice.add_account(BankAccount("Alice", 100))
alice.add_account(BankAccount("Alice", 250))

print(alice)
print("Total balance:", alice.total_balance())

---

# 8) Domain modeling: Product / Order

We create classes that mirror a business domain:
- Product: name, price
- Order: collection of items (product + quantity)

Key design decision:
- Store items as a list of tuples `(product, qty)` (simple and explicit).
Later we can refactor to a dedicated `OrderItem` class.

In [ ]:
class Product:
    def __init__(self, name, price):
        if not name:
            raise ValueError("Product name must be non-empty")
        price = float(price)
        if price < 0:
            raise ValueError("Price must be >= 0")
        self.name = name
        self.price = price

    def __repr__(self):
        return f"Product(name={self.name!r}, price={self.price:.2f})"

class Order:
    def __init__(self):
        self.items = []  # list of (Product, quantity)

    def add(self, product, quantity=1):
        if not isinstance(product, Product):
            raise TypeError("product must be a Product")
        quantity = int(quantity)
        if quantity <= 0:
            raise ValueError("quantity must be positive")
        self.items.append((product, quantity))

    def total(self):
        return sum(p.price * q for p, q in self.items)

    def receipt_lines(self):
        lines = []
        for p, q in self.items:
            lines.append(f"{p.name:12s} x{q:2d}  {p.price*q:8.2f}")
        lines.append("-" * 26)
        lines.append(f"{'TOTAL':12s}      {self.total():8.2f}")
        return lines

    def print_receipt(self):
        print("RECEIPT")
        for line in self.receipt_lines():
            print(line)

apple = Product("Apple", 0.50)
banana = Product("Banana", 0.30)
milk = Product("Milk", 1.20)

order = Order()
order.add(apple, 4)
order.add(banana, 10)
order.add(milk, 1)

print(order.total())
order.print_receipt()

---

# 9) Basic inheritance

Inheritance lets you reuse code and extend behavior.

- Base class: general behavior
- Child class: specialized behavior

Use inheritance when there is an **"is-a"** relationship:
- SavingsAccount *is an* Account
- Manager *is an* Employee

In [ ]:
class Account:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit must be positive")
        self.balance += amount

    def withdraw(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdraw must be positive")
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

    def __repr__(self):
        return f"Account(owner={self.owner!r}, balance={self.balance:.2f})"

class SavingsAccount(Account):
    def apply_interest(self, rate):
        rate = float(rate)
        if rate < 0:
            raise ValueError("rate must be >= 0")
        self.balance += self.balance * rate

sa = SavingsAccount("Alice", 1000)
sa.deposit(200)
sa.apply_interest(0.05)
print(sa)

### Overriding methods (polymorphism, in a basic form)

A child class can override a method from the base class.

Example: a `FeeAccount` charges a fixed fee on each withdrawal.

In [ ]:
class FeeAccount(Account):
    def __init__(self, owner, balance=0.0, fee=1.0):
        super().__init__(owner, balance)
        self.fee = float(fee)

    def withdraw(self, amount):
        # Charge fee on top of amount
        total = float(amount) + self.fee
        return super().withdraw(total)

fa = FeeAccount("Bob", 100, fee=2.5)
fa.withdraw(10)
print(fa)

---

# 10) Common OOP mistakes (and how to avoid them)

1. **Forgetting `self`** in methods:
   - `def deposit(amount):` is wrong
2. **Using class attributes when you meant instance attributes**
3. **Mixing I/O with logic everywhere**
4. Creating “God objects” that do too many responsibilities
5. Overusing inheritance where composition is simpler

In [ ]:
# Example: class attribute vs instance attribute

class Bag:
    items = []  # WARNING: class attribute shared by all instances!

b1 = Bag()
b2 = Bag()
b1.items.append("apple")

print("b1:", b1.items)
print("b2:", b2.items)  # surprise: shared list

Correct approach: create the list per instance in `__init__`.

In [ ]:
class Bag:
    def __init__(self):
        self.items = []

b1 = Bag()
b2 = Bag()
b1.items.append("apple")

print("b1:", b1.items)
print("b2:", b2.items)

---

# Practice (Exercises)

The exercises are designed to reinforce:
- state changes via methods,
- validation,
- composition,
- inheritance.

You can solve them directly below.

## Exercise 1 — BankAccount with validation and transfer

**Task:**
Implement `BankAccount2` with:
- `deposit(amount)`
- `withdraw(amount)`
- `transfer_to(other, amount)`
- validation (no negative, no overdraft)

Add a short demo at the bottom.

In [ ]:
# Exercise 1 (starter)

class BankAccount2:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = float(balance)

    def deposit(self, amount):
        # TODO
        pass

    def withdraw(self, amount):
        # TODO
        pass

    def transfer_to(self, other, amount):
        # TODO
        pass

# Demo (after implementing):
# a = BankAccount2("Alice", 100)
# b = BankAccount2("Bob", 50)
# a.transfer_to(b, 30)
# print(a.balance, b.balance)

## Exercise 2 — Order enhancements

**Task:**
Extend the `Order` class to support:
- removing a product (by name or by object)
- computing a discounted total (e.g. 10% off)
- printing a receipt with optional discount

Hint: you may find it easier to store items in a dict:
`{product: quantity}` or `{product_name: (product, quantity)}`

In [ ]:
# Exercise 2 (starter)

class Order2:
    def __init__(self):
        self.items = []  # you may change this structure

    def add(self, product, quantity=1):
        pass

    def remove(self, product_name):
        pass

    def total(self, discount_rate=0.0):
        pass

    def print_receipt(self, discount_rate=0.0):
        pass

## Exercise 3 — Inheritance: Employee / Manager

**Task:**
Create:
- class `Employee(name, salary)`
- method `total_compensation()`
- subclass `Manager(name, salary, bonus)`
  - overrides `total_compensation()`

Create 1 employee and 1 manager and print both compensations.

In [ ]:
# Exercise 3 (starter)

class Employee:
    def __init__(self, name, salary):
        self.name = name
        self.salary = float(salary)

    def total_compensation(self):
        # TODO
        pass

class Manager(Employee):
    def __init__(self, name, salary, bonus):
        super().__init__(name, salary)
        self.bonus = float(bonus)

    def total_compensation(self):
        # TODO
        pass

# Demo:
# e = Employee("Alice", 50000)
# m = Manager("Bob", 70000, 10000)
# print(e.total_compensation())
# print(m.total_compensation())

---

## Checklist for Unit 7

You should now be comfortable with:

- [ ] Class vs object, instance creation
- [ ] Attributes and methods (`self`)
- [ ] Constructors (`__init__`) and valid object state
- [ ] Modeling state and behavior (e.g., accounts, orders)
- [ ] Composition (objects containing objects)
- [ ] Basic inheritance and method overriding
- [ ] Avoiding common pitfalls (shared class attributes)

Next: **Unit 8 — Advanced OOP & Design Principles**
(encapsulation, properties, abstract classes, composition vs inheritance, patterns).